In [2]:
import folium
import pandas as pd
import numpy as np

In [4]:
data = pd.read_csv('./zillow.csv')
data.head()

,latitude,longtitude,floorSize,url,price
0,34.125356,-118.177763,"2,090",https://www.zillow.com/homedetails/1020-Lantan...,"$1,300,000"
1,34.206636,-118.593988,"1,282",https://www.zillow.com/homedetails/7502-Milwoo...,"$709,000"
2,34.110423,-118.375414,"2,207",https://www.zillow.com/homedetails/2190-Beech-...,"$2,699,000"
3,33.982543,-118.288348,"1,218",https://www.zillow.com/homedetails/836-W-62nd-...,"$405,598"
4,34.057765,-118.356241,"4,616",https://www.zillow.com/homedetails/5809-W-Olym...,"$1,750,000"


In [6]:
latitude = 34.0522
longitude = -118.2437
# Create map and display it
la_map = folium.Map(location=[latitude, longitude], zoom_start=12)

# Display the map of LA
la_map

In [8]:
limit = 200
ldata = data.iloc[0:limit, :]
incidents = folium.map.FeatureGroup()

for lat,lng in zip(ldata.latitude, ldata.longtitude):
    incidents.add_child(
        folium.CircleMarker(
            [lat, lng],
            radius=7,
            color = 'yellow',
            fill = True,
            fill_color='red'
            # fill_opacity=0.4
        )
    )

la_map = folium.Map(location=[latitude, longitude], zoom_start=12)
la_map.add_child(incidents)

In [10]:
from folium import plugins

# let's start again with a clean copy of the map of San Francisco
la_map = folium.Map(location = [latitude, longitude], zoom_start = 12)

# instantiate a mark cluster object for the incidents in the dataframe
incidents = plugins.MarkerCluster().add_to(la_map)

# loop through the dataframe and add each data point to the mark cluster
for lat, lng, label, in zip(data.latitude, data.longtitude, data.price):
    folium.Marker(
        location=[lat, lng],
        icon=None,
        popup=label
    ).add_to(incidents)

# add incidents to map
la_map.add_child(incidents)

In [12]:
import json
import requests

url = 'http://s3-us-west-2.amazonaws.com/boundaries.latimes.com/archive/1.0/boundary-set/la-county-neighborhoods-v6.geojson'
la_geo = f'{url}'
la_map = folium.Map(location=[latitude, longitude], zoom_start=12)
folium.GeoJson(
    la_geo,
    style_function=lambda feature: {
        'fillColor': '#ffff00',
        'color': 'black',
        'weight': 2,
        'dashArray': '5, 5'
    }
).add_to(la_map)

#display map
la_map

In [14]:
def convert_currency(value):
    """
    转换字符串数字为float类型
     - 移除 ￥ ,
     - 转化为float类型
    """
    new_value = value.replace(',', '').replace('$', '')
    return np.float(new_value)
